In [1]:
#dependencies
import re
from bs4 import BeautifulSoup 
from splinter import Browser
import requests
import pandas as pd
from selenium import webdriver 
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service as BraveService
from webdriver_manager.core.utils import ChromeType
import json
import time
from tqdm.notebook import tqdm 

<h4> Start cells here if partial FIPS already available in {state}_fips_scraped file

In [2]:
fips_check_scraped = pd.read_csv("../../../data/state_data/geo/geo_fips/23/AZ_fips_scraped.csv") 
fips_check_scraped  = fips_check_scraped.loc[:, ~fips_check_scraped.columns.str.contains('^Unnamed')] 
fips_check_scraped.head() 

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,2008847101,AZ,"9366 East Raintree Drive, SCOTTSDALE, AZ",33.620398,-111.878704,4.013217e+13
1,3057347105,AZ,"PO Box 170, FORT DEFIANCE, AZ",35.744460,-109.076483,4.001944e+13
2,3470877104,AZ,"2999 N44th Street Suite 100, Phoenix, AZ",33.482323,-111.986368,4.013111e+13
3,3530847305,AZ,"5055 E BROADWAY BLVD Ste A100, TUCSON, AZ",32.222108,-110.884676,4.019003e+13
4,3771367108,AZ,"8880 E CHAPPARAL RD SUITE 300, SCOTTSDALE, AZ",33.509368,-111.889853,4.013941e+13


In [3]:
#Latitude, Longitude, not included
fips_check_scraped_notna = fips_check_scraped[fips_check_scraped["Lat"].notna()] # check to make sure no nulls in coords
fips_check_scraped_notna.head() 

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,2008847101,AZ,"9366 East Raintree Drive, SCOTTSDALE, AZ",33.620398,-111.878704,4.013217e+13
1,3057347105,AZ,"PO Box 170, FORT DEFIANCE, AZ",35.744460,-109.076483,4.001944e+13
2,3470877104,AZ,"2999 N44th Street Suite 100, Phoenix, AZ",33.482323,-111.986368,4.013111e+13
3,3530847305,AZ,"5055 E BROADWAY BLVD Ste A100, TUCSON, AZ",32.222108,-110.884676,4.019003e+13
4,3771367108,AZ,"8880 E CHAPPARAL RD SUITE 300, SCOTTSDALE, AZ",33.509368,-111.889853,4.013941e+13


In [4]:
#FIPS not included, of Lat, Long available
fips_check_scraped_null = fips_check_scraped_notna[fips_check_scraped_notna["FIPS_z"].isnull()] # check to make sure no nulls in coords
fips_check_scraped_null #28093

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
727,3445788509,AZ,"PRIDE BAKERIES LLC, PHOENIX, AZ",38.740566,-90.297851,NaN
947,5478438406,AZ,"18716 S Gadsden St, Gadsden, AZ",32.555962,-114.783982,NaN
969,5363168310,AZ,"65 AVENIDA DE OTERO, TUBAC, AZ",31.625668,-111.043712,NaN
1161,9593207001,AZ,"3200 Willow Creek Rd, PRESCOTT, AZ",34.605471,-112.455316,NaN
1891,9950128410,AZ,"7228 E 1ST AVENUE, SCOTTSDALE, AZ",33.494246,-111.925363,NaN
3482,5800237310,AZ,"1/4 miles N. Antelope Rd N22B, Page, AZ",36.912977,-111.449174,NaN
4049,4410207007,AZ,"P.O. Box 19460, PHOENIX, AZ",40.118071,-75.539316,NaN
4081,1641998609,AZ,"1762 Highway 95, Bullhead City, AZ",43.844102,-116.953670,NaN
5468,8620287005,AZ,"6240 Tacoma Mall Blvd. Ste 315, QUEEN CREEK, AZ",47.199526,-122.463532,NaN
5706,8003008504,AZ,"1/4 mile N. Antelope Rd N22B, Page, AZ",36.912977,-111.449174,NaN


<h4>FIPS Script

In [5]:
#if FIPS_z file 
lat = fips_check_scraped_null["Lat"].astype(float).astype(str).to_list()
long = fips_check_scraped_null["Long"].astype(float).astype(str).to_list()

In [6]:
#before running script, check for any NaN within Lat, Long fields. 
def sleep():
    time.sleep(1)
option = webdriver.ChromeOptions()
option.binary_location = "/Applications/Brave Browser.app/Contents/MacOS/Brave Browser"
driver = Browser('chrome', options=option, headless=False)    
url = 'https://geocoding.geo.census.gov/geocoder/geographies/coordinates?form'
driver.visit(url)

not_found = []
geo_id = []

i=0
for i in tqdm(range(len(lat))):
    #while i < len(lat):
        driver.find_by_id('geoCoordX').clear()
        driver.find_by_id('geoCoordY').clear()
        driver.find_by_id('geoCoordX').fill(long[i])
        driver.find_by_id('geoCoordY').fill(lat[i])
        driver.find_by_id('geoCoordsSubmit').click()
        #sleep()
        #html = driver.html
        soup = BeautifulSoup(driver.html, 'html.parser')


        results = soup.find_all('div', id='geoCoordsJsonOutput')
        for result in results:
            result_text = result.text

            result_digits = re.findall('\d+', result_text)
            startswith = [s for s in result_digits if s.startswith('04')] #AZ
            fips_elements =[element for element in startswith if len(element)>=15]
            
            try:
                #print(f"{i},{lat[i]},{long[i]} is {fips_elements}")
                geo_id.append(fips_elements)
            except (KeyError, IndexError):
                #print(f"{i},{lat[i]},{long[i]} Not Found")
                not_found.append(i)
                geo_id.append(None)  
            except (TimeoutException):
                driver.navigate.refresh()
    
        #sleep()
        i=i+1

  0%|          | 0/16 [00:00<?, ?it/s]

coordinates wtih more than one return:
385,37.514070000000004,-77.35472 is ['514070000000004', '510872015041003'] 
1178,37.516454700000004,-77.8100575 is ['516454700000004', '511455001022014'] 
2660,37.519462299999994,-77.340384 is ['519462299999994', '510872014061012']
second option reference FIPS code, first option references LatCoord, update fields in DataFrame

https://iqss.github.io/dss-webscrape/filling-in-web-forms.html

In [7]:
geo_id_list =geo_id 

Length Check

In [8]:
print(len(geo_id_list))

16


Matching FIPS to DataFrame

In [9]:
fips_fill = fips_check_scraped_null

In [10]:
fips_fill["FIPS_z"] = geo_id_list
fips_fill

/var/folders/vy/qdp8xcfj4s3dd5c6d1xp_y3w0000gn/T/ipykernel_13111/597263667.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fips_fill["FIPS_z"] = geo_id_list


,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
727,3445788509,AZ,"PRIDE BAKERIES LLC, PHOENIX, AZ",38.740566,-90.297851,[]
947,5478438406,AZ,"18716 S Gadsden St, Gadsden, AZ",32.555962,-114.783982,[040270115011056]
969,5363168310,AZ,"65 AVENIDA DE OTERO, TUBAC, AZ",31.625668,-111.043712,[040239661012009]
1161,9593207001,AZ,"3200 Willow Creek Rd, PRESCOTT, AZ",34.605471,-112.455316,[040250003012018]
1891,9950128410,AZ,"7228 E 1ST AVENUE, SCOTTSDALE, AZ",33.494246,-111.925363,[040132172012032]
3482,5800237310,AZ,"1/4 miles N. Antelope Rd N22B, Page, AZ",36.912977,-111.449174,[040050021014014]
4049,4410207007,AZ,"P.O. Box 19460, PHOENIX, AZ",40.118071,-75.539316,[]
4081,1641998609,AZ,"1762 Highway 95, Bullhead City, AZ",43.844102,-116.953670,[]
5468,8620287005,AZ,"6240 Tacoma Mall Blvd. Ste 315, QUEEN CREEK, AZ",47.199526,-122.463532,[]
5706,8003008504,AZ,"1/4 mile N. Antelope Rd N22B, Page, AZ",36.912977,-111.449174,[040050021014014]


Remove brackets

In [11]:
fips_fill['FIPS_z'] = fips_fill['FIPS_z'].str[0]
fips_fill

/var/folders/vy/qdp8xcfj4s3dd5c6d1xp_y3w0000gn/T/ipykernel_13111/1219885402.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fips_fill['FIPS_z'] = fips_fill['FIPS_z'].str[0]


,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
727,3445788509,AZ,"PRIDE BAKERIES LLC, PHOENIX, AZ",38.740566,-90.297851,NaN
947,5478438406,AZ,"18716 S Gadsden St, Gadsden, AZ",32.555962,-114.783982,040270115011056
969,5363168310,AZ,"65 AVENIDA DE OTERO, TUBAC, AZ",31.625668,-111.043712,040239661012009
1161,9593207001,AZ,"3200 Willow Creek Rd, PRESCOTT, AZ",34.605471,-112.455316,040250003012018
1891,9950128410,AZ,"7228 E 1ST AVENUE, SCOTTSDALE, AZ",33.494246,-111.925363,040132172012032
3482,5800237310,AZ,"1/4 miles N. Antelope Rd N22B, Page, AZ",36.912977,-111.449174,040050021014014
4049,4410207007,AZ,"P.O. Box 19460, PHOENIX, AZ",40.118071,-75.539316,NaN
4081,1641998609,AZ,"1762 Highway 95, Bullhead City, AZ",43.844102,-116.953670,NaN
5468,8620287005,AZ,"6240 Tacoma Mall Blvd. Ste 315, QUEEN CREEK, AZ",47.199526,-122.463532,NaN
5706,8003008504,AZ,"1/4 mile N. Antelope Rd N22B, Page, AZ",36.912977,-111.449174,040050021014014


In [12]:
fips_fill.loc[fips_fill["FIPS_z"].isnull()] #check #13

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
727,3445788509,AZ,"PRIDE BAKERIES LLC, PHOENIX, AZ",38.740566,-90.297851,NaN
4049,4410207007,AZ,"P.O. Box 19460, PHOENIX, AZ",40.118071,-75.539316,NaN
4081,1641998609,AZ,"1762 Highway 95, Bullhead City, AZ",43.844102,-116.953670,NaN
5468,8620287005,AZ,"6240 Tacoma Mall Blvd. Ste 315, QUEEN CREEK, AZ",47.199526,-122.463532,NaN
6134,4214117203,AZ,"1762 HIGHWAY 95, Bullhead City, AZ",43.844102,-116.953670,NaN
15066,9476237708,AZ,"MILE POST 1215 US 95 N, COURT SIDE, AZ",34.228172,-114.635898,NaN


Parse for Merge

In [13]:
cols= ["LoanNumber", "FIPS_z"]
fips_fill_parsed = fips_fill[cols]
fips_fill_parsed

,LoanNumber,FIPS_z
727,3445788509,NaN
947,5478438406,040270115011056
969,5363168310,040239661012009
1161,9593207001,040250003012018
1891,9950128410,040132172012032
3482,5800237310,040050021014014
4049,4410207007,NaN
4081,1641998609,NaN
5468,8620287005,NaN
5706,8003008504,040050021014014


Read in base data

In [14]:

#if from FIPS file
fips_check = pd.read_csv("../../../data/state_data/geo/geo_fips/23/AZ_fips_scraped.csv") 
#fips_check = fips_check.drop(["Unnamed: 0.1", "Unnamed: 0.1.1", "Unnamed: 0"], axis =1) 
fips_check.head()

,Unnamed: 0,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,19451,2008847101,AZ,"9366 East Raintree Drive, SCOTTSDALE, AZ",33.620398,-111.878704,4.013217e+13
1,19452,3057347105,AZ,"PO Box 170, FORT DEFIANCE, AZ",35.744460,-109.076483,4.001944e+13
2,19453,3470877104,AZ,"2999 N44th Street Suite 100, Phoenix, AZ",33.482323,-111.986368,4.013111e+13
3,19454,3530847305,AZ,"5055 E BROADWAY BLVD Ste A100, TUCSON, AZ",32.222108,-110.884676,4.019003e+13
4,19455,3771367108,AZ,"8880 E CHAPPARAL RD SUITE 300, SCOTTSDALE, AZ",33.509368,-111.889853,4.013941e+13


Merge with MainFrame

In [15]:
fips_merge = fips_check.merge(fips_fill_parsed, on="LoanNumber", how="left")
fips_merge

,Unnamed: 0,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z_x,FIPS_z_y
0,19451,2008847101,AZ,"9366 East Raintree Drive, SCOTTSDALE, AZ",33.620398,-111.878704,4.013217e+13,NaN
1,19452,3057347105,AZ,"PO Box 170, FORT DEFIANCE, AZ",35.744460,-109.076483,4.001944e+13,NaN
2,19453,3470877104,AZ,"2999 N44th Street Suite 100, Phoenix, AZ",33.482323,-111.986368,4.013111e+13,NaN
3,19454,3530847305,AZ,"5055 E BROADWAY BLVD Ste A100, TUCSON, AZ",32.222108,-110.884676,4.019003e+13,NaN
4,19455,3771367108,AZ,"8880 E CHAPPARAL RD SUITE 300, SCOTTSDALE, AZ",33.509368,-111.889853,4.013941e+13,NaN
...,...,...,...,...,...,...,...,...
15752,35203,9422327705,AZ,"7164 38TH ST, TUCSON, AZ",32.189429,-110.839316,4.019004e+13,NaN
15753,35204,9650217306,AZ,"1933 E Janice Way, Phoenix, AZ",33.623513,-112.041885,4.013104e+13,NaN
15754,35205,9716018610,AZ,"1927 N Gilbert Rd Ste 7, Mesa, AZ",33.450479,-111.787078,4.013421e+13,NaN
15755,35206,9821317002,AZ,"235 E BASELINE RD, TEMPE, AZ",33.377459,-111.935714,4.013320e+13,NaN


<b>Fill FIPS Columns - if FIPS file

In [16]:
fips_merge["FIPS_z"] = fips_merge["FIPS_z_y"].fillna(fips_merge["FIPS_z_x"])
fips_merge.head()

,Unnamed: 0,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z_x,FIPS_z_y,FIPS_z
0,19451,2008847101,AZ,"9366 East Raintree Drive, SCOTTSDALE, AZ",33.620398,-111.878704,4.013217e+13,NaN,40132168501000.0
1,19452,3057347105,AZ,"PO Box 170, FORT DEFIANCE, AZ",35.744460,-109.076483,4.001944e+13,NaN,40019440002055.0
2,19453,3470877104,AZ,"2999 N44th Street Suite 100, Phoenix, AZ",33.482323,-111.986368,4.013111e+13,NaN,40131110004009.0
3,19454,3530847305,AZ,"5055 E BROADWAY BLVD Ste A100, TUCSON, AZ",32.222108,-110.884676,4.019003e+13,NaN,40190033043006.0
4,19455,3771367108,AZ,"8880 E CHAPPARAL RD SUITE 300, SCOTTSDALE, AZ",33.509368,-111.889853,4.013941e+13,NaN,40139413001046.0


Drop Extra FIPS Columns

In [17]:
fips_merge_drop = fips_merge.drop(["FIPS_z_x", "FIPS_z_y"], axis=1)
fips_merge_drop = fips_merge_drop.drop(["Unnamed: 0"], axis=1)
fips_merge_drop

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,2008847101,AZ,"9366 East Raintree Drive, SCOTTSDALE, AZ",33.620398,-111.878704,40132168501000.0
1,3057347105,AZ,"PO Box 170, FORT DEFIANCE, AZ",35.744460,-109.076483,40019440002055.0
2,3470877104,AZ,"2999 N44th Street Suite 100, Phoenix, AZ",33.482323,-111.986368,40131110004009.0
3,3530847305,AZ,"5055 E BROADWAY BLVD Ste A100, TUCSON, AZ",32.222108,-110.884676,40190033043006.0
4,3771367108,AZ,"8880 E CHAPPARAL RD SUITE 300, SCOTTSDALE, AZ",33.509368,-111.889853,40139413001046.0
...,...,...,...,...,...,...
15752,9422327705,AZ,"7164 38TH ST, TUCSON, AZ",32.189429,-110.839316,40190040372003.0
15753,9650217306,AZ,"1933 E Janice Way, Phoenix, AZ",33.623513,-112.041885,40131036074002.0
15754,9716018610,AZ,"1927 N Gilbert Rd Ste 7, Mesa, AZ",33.450479,-111.787078,40134206031017.0
15755,9821317002,AZ,"235 E BASELINE RD, TEMPE, AZ",33.377459,-111.935714,40133199021002.0


In [18]:
fips_merge_drop.loc[fips_merge_drop["FIPS_z"].isnull()] #13

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
727,3445788509,AZ,"PRIDE BAKERIES LLC, PHOENIX, AZ",38.740566,-90.297851,NaN
4049,4410207007,AZ,"P.O. Box 19460, PHOENIX, AZ",40.118071,-75.539316,NaN
4081,1641998609,AZ,"1762 Highway 95, Bullhead City, AZ",43.844102,-116.953670,NaN
5468,8620287005,AZ,"6240 Tacoma Mall Blvd. Ste 315, QUEEN CREEK, AZ",47.199526,-122.463532,NaN
6134,4214117203,AZ,"1762 HIGHWAY 95, Bullhead City, AZ",43.844102,-116.953670,NaN
15066,9476237708,AZ,"MILE POST 1215 US 95 N, COURT SIDE, AZ",34.228172,-114.635898,NaN


Convert to File

In [19]:
fips_AZ = fips_merge_drop

In [20]:
fips_AZ.to_csv("../../../data/state_data/geo/geo_fips/23/AZ_fips_scraped.csv") 